In [1]:
using JuMP
using Ipopt
using CSV, DataFrames, ProgressMeter, Distributions

In [2]:
using Interpolations

In [3]:
# 船体に働く流体力
function X_H(u, v, r)
    U = sqrt(u^2 + (v - r * x_G)^2)
    β = asin(-(v - r * x_G) / U)
    r_dash = r * L_pp / U
    v_dash = v / U
    return 0.5 * ρ * L_pp * d * U^2 * X_H_dash(v_dash, r_dash)
end

function Y_H(u, v, r)
    U = sqrt(u^2 + (v - r * x_G)^2)
    β = asin(-(v - r * x_G) / U)
    r_dash = r * L_pp / U
    v_dash = v / U
    return 0.5 * ρ * L_pp * d * U^2 * Y_H_dash(v_dash, r_dash)
end

function N_H(u, v, r)
    U = sqrt(u^2 + (v - r * x_G)^2)
    β = asin(-(v - r * x_G) / U)
    r_dash = r * L_pp / U
    v_dash = v / U
    return 0.5 * ρ * L_pp^2 * d * U^2 * N_H_dash(v_dash, r_dash)
end

# 船体に働く流体力(無次元)
function X_H_dash(β, r_dash)
    return -R_0_dash + X_vv_dash*β^2 + X_vr_dash*β*r_dash + X_rr_dash*r_dash^2 + X_vvvv_dash*β^4
end

function Y_H_dash(β, r_dash)
    return Y_v_dash*β + Y_r_dash*r_dash + Y_vvr_dash*β^2*r_dash + Y_vrr_dash*β*r_dash^2 + Y_vvv_dash*β^3 + Y_rrr_dash*r_dash^3
end

function N_H_dash(β, r_dash)
    return N_v_dash*β + N_r_dash*r_dash + N_vvr_dash*β^2*r_dash + N_vrr_dash*β*r_dash^2 + N_vvv_dash*β^3 + N_rrr_dash*r_dash^3
end

# 舵に働く力
function X_R(u, v, r, δ, n_p)
    return -(1 - t_R) * F_N(u, v, r, δ, n_p) * sin(δ)
end

function Y_R(u, v, r, δ, n_p)
    return -(1 + a_H) * F_N(u, v, r, δ, n_p) * cos(δ)
end

function N_R(u, v, r, δ, n_p)
    return -(x_R + a_H * x_H) * F_N(u, v, r, δ, n_p) * cos(δ)
end

function F_N(u, v, r, δ, n_p)
    U = sqrt(u^2 + (v - r * x_G)^2)
    β = asin(-(v - r * x_G) / U)
    r_dash = r * L_pp / U
    w_P = w_P0 * exp(-4.0 * (β - x_P * r_dash)^2)
    u_p = (1 - w_P) * u
    u_R = ϵ * u_p * sqrt(η * (1 + κ * (sqrt(1 + (8 * K_T(u, v, r, n_p)) / (π * J(u, v, r, n_p)^2)) - 1))^2 + (1 - η))
    β_R = β - l_R * r_dash
    if β_R < 0.0
        γ_R = γ_R_minus
    else
        γ_R = γ_R_plus
    end 
    v_R = U * γ_R * (β - l_r_dash * r_dash)
    α_R = δ - atan(v_R, u_R)
    U_R = sqrt(u_R^2 + v_R^2)
    return 0.5 * ρ * A_R * U_R^2 * f_α * sin(α_R)
end

# プロペラに働く力
function X_P(u, v, r, δ, n_p)
    return (1 - t_P) * T_P(u, v, r, n_p)
end

function K_T(u, v, r, n_p)
    return k_0 + k_1 * J(u, v, r, n_p) + k_2 * J(u, v, r, n_p)^2
end

function J(u, v, r, n_p)
    U = sqrt(u^2 + (v - r * x_G)^2)
    β = atan(-v, u)
    r_dash = r * L_pp / U
    w_P = w_P0 * exp(-4.0 * (β - x_P * r_dash)^2)
    return u * (1 - w_P) / (n_p * D_p)
end

function T_P(u, v, r, n_p)
    return K_T(u, v, r, n_p) * ρ * n_p^2 * D_p^4
end

T_P (generic function with 1 method)

In [4]:
# パラメータを定義
const ρ = 1025.0
const L_pp = 3.50
const B = 0.57  
const d = 0.16  
const nabla = 0.7407 * L_pp * B * d 
const x_G = 0.0112 
const D_p = 0.03373 * L_pp 
const m = nabla * ρ 
const I_zG = m * ((0.25 * L_pp)^2)
const A_R = 0.0189  
const η = 0.8023  
const m_x = 0.0475 * m 
const m_y = 0.6739 * m 
const J_z = 0.0306 * m * (L_pp^2) 
const f_α = 2.634 
const ϵ = 1.345
const t_R = 0.084
const a_H = 0.054
const x_H = -0.388 * L_pp
const γ_R_minus = 0.225 
const γ_R_plus = 0.574 
const l_r_dash = -0.641 
const l_R = -0.641  
const κ = 0.482
const t_P = 0.180 
const w_P0 = 0.40  
const x_P = -0.490 
const x_R = -0.500 * L_pp

const k_0 = 0.3516
const k_1 = -0.3624
const k_2 = -0.1100

-0.11

In [5]:
R_0_dash = 0.020
X_vv_dash = -0.0799
X_vr_dash = 0.1016
X_rr_dash = -0.0176
X_vvvv_dash = 0.5704
Y_v_dash = -0.3711
Y_r_dash = 0.0720
Y_vvv_dash = -1.0535
Y_vvr_dash = 0.207
Y_vrr_dash = -0.2556
Y_rrr_dash = 0.0220
N_v_dash = -0.1097
N_r_dash = -0.0527
N_vvv_dash = -0.0533
N_vvr_dash = -0.2863
N_vrr_dash = -0.0206
N_rrr_dash = -0.0196

-0.0196

In [6]:
function X_wind(u, v, ψ)
    ρ_air = 1.225
    
    # 相対風速の計算
    u_A = -U_W * cos(Ψ_W) - u * cos(ψ) + v * sin(ψ)
    v_A = U_W * sin(Ψ_W) - u * sin(ψ) - v * cos(ψ)
    
    # 風速の計算
    if U_W == 0.0
        U_A = 0.0
    else
        U_A = sqrt(u_A^2 + v_A^2)
    end
    
    # 相対風向の計算
    Ψ_A = ψ - Ψ_W
    Ψ_A = mod(Ψ_A, 2 * π)
    
    # 風圧力の計算
    return ρ_air * A_F * C_X(Ψ_A) / 2 * U_A^2
end

X_wind (generic function with 1 method)

In [7]:
function Y_wind(u, v, ψ)
    ρ_air = 1.225
    
    # 相対風速の計算
    u_A = -U_W * cos(Ψ_W) - u * cos(ψ) + v * sin(ψ)
    v_A = U_W * sin(Ψ_W) - u * sin(ψ) - v * cos(ψ)
    
    # 風速の計算
    if U_W == 0.0
        U_A = 0.0
    else
        U_A = sqrt(u_A^2 + v_A^2)
    end
    
    # 相対風向の計算
    Ψ_A = ψ - Ψ_W
    Ψ_A = mod(Ψ_A, 2 * π)
   
    # 風圧力の計算
    return ρ_air * A_L * C_Y(Ψ_A) / 2 * U_A^2
end

Y_wind (generic function with 1 method)

In [8]:
function N_wind(u, v, ψ)
    ρ_air = 1.225
    
    # 相対風速の計算
    u_A = -U_W * cos(Ψ_W) - u * cos(ψ) + v * sin(ψ)
    v_A = U_W * sin(Ψ_W) - u * sin(ψ) - v * cos(ψ)
    
    # 風速の計算
    if U_W == 0.0
        U_A = 0.0
    else
        U_A = sqrt(u_A^2 + v_A^2)
    end
    
    # 相対風向の計算
    Ψ_A = ψ - Ψ_W
    Ψ_A = mod(Ψ_A, 2 * π)
       
    # 風圧力の計算
    return ρ_air * A_L * L_pp * C_N(Ψ_A) / 2 * U_A^2
end

N_wind (generic function with 1 method)

In [ ]:
function mpc(f_x_t::Vector{Float64}, u1_t::Float64, u2_t::Float64, u3_t::Float64, f_obs::DataFrame ,t::Int, N::Int)
    global R_0_dash, X_vv_dash, X_vr_dash, X_rr_dash, X_vvvv_dash
    global Y_v_dash, Y_r_dash, Y_vvv_dash, Y_vvr_dash, Y_vrr_dash, Y_rrr_dash
    global N_v_dash, N_r_dash, N_vvv_dash, N_vvr_dash, N_vrr_dash, N_rrr_dash

    mcmc = CSV.read("./observation/MCMC_L2_20deg.csv", DataFrame)
    # モデル作成
    model = Model(Ipopt.Optimizer) # Ipoptを使用
    set_silent(model) # 計算の進行状況を非表示
    L_f = 1.0 # センサー位置(船首)
    L_t = 1.0 # センサー位置(船尾)
    Q = 10000.0 # 座標誤差への重み
    R1 = 10.0 # 制御量変化(X_F)への重み
    R2 = 10.0# 制御量変化(Y_F)への重み
    R3 = 1.0# 制御量変化(N_F)への重み
    δ_ref =  20 * pi / 180.0 # [rad]
    n_p_ref = 21.21824 # [rps]

    # 方程式を登録
    register(model, :X_H, 3, X_H; autodiff = true)
    register(model, :Y_H, 3, Y_H; autodiff = true)
    register(model, :N_H, 3, N_H; autodiff = true)
    register(model, :X_H_dash, 2, X_H_dash; autodiff = true)
    register(model, :Y_H_dash, 2, Y_H_dash; autodiff = true)
    register(model, :N_H_dash, 2, N_H_dash; autodiff = true)
    register(model, :X_R, 5, X_R; autodiff = true)
    register(model, :Y_R, 5, Y_R; autodiff = true)
    register(model, :N_R, 5, N_R; autodiff = true)
    register(model, :F_N, 5, F_N; autodiff = true)
    register(model, :X_P, 5, X_P; autodiff = true)
    register(model, :K_T, 4, K_T; autodiff = true)
    register(model, :J, 4, J; autodiff = true)
    register(model, :T_P, 4, T_P; autodiff = true)
    register(model, :X_wind, 3, X_wind; autodiff = true)
    register(model, :Y_wind, 3, Y_wind; autodiff = true)
    register(model, :N_wind, 3, N_wind; autodiff = true)    

    # 使用する変数を設定
    x = @variable(model, [t:t+N+1, 1:13]) # 操作量 x
    u1 = @variable(model, [t:t+N, 1:1]) # 制御量 u1
    u2 = @variable(model, [t:t+N, 1:1]) # 制御量 u2
    u3 = @variable(model, [t:t+N, 1:1])

    # 制御量の制約条件
    @constraint(model, -10.0 .<= u1[t:t+N, 1:1] .<= 10.0) 
    @constraint(model, -10.0 .<= u2[t:t+N, 1:1] .<= 10.0)
    @constraint(model, -10.0 .<= u3[t:t+N, 1:1] .<= 10.0)

    # 操作量の箱を作成
    for k in t:t+N+1
        set_start_value.(x[k, 1], f_x_t[1])
        set_start_value.(x[k, 2], f_x_t[2])
        set_start_value.(x[k, 3], f_x_t[3])
        set_start_value.(x[k, 4], f_x_t[4])
        set_start_value.(x[k, 5], f_x_t[5])
        set_start_value.(x[k, 6], f_x_t[6])
        set_start_value.(x[k, 7], f_x_t[7])
        set_start_value.(x[k, 8], f_x_t[8])
        set_start_value.(x[k, 9], f_x_t[9])
        set_start_value.(x[k, 10], f_x_t[10])
        set_start_value.(x[k, 11], f_x_t[11])
        set_start_value.(x[k, 12], f_x_t[12])
        set_start_value.(x[k, 13], f_x_t[13])   
    end
    
    # 初期状態(t)での変数xの設定
    for i in 1:13
        @constraint(model, x[t, i] == f_x_t[i])
    end

    # MMGを用いた制約
    for k in t:t+N
        random_num = rand(500:1000)
        R_0_dash = mcmc[random_num,4]
        X_vv_dash = mcmc[random_num,5]
        X_vr_dash = mcmc[random_num,6]
        X_rr_dash = mcmc[random_num,7]
        X_vvvv_dash = mcmc[random_num,8]
        Y_v_dash = mcmc[random_num,9]
        Y_r_dash = mcmc[random_num,10]
        Y_vvv_dash = mcmc[random_num,11]
        Y_vvr_dash = mcmc[random_num,12]
        Y_vrr_dash = mcmc[random_num,13]
        Y_rrr_dash = mcmc[random_num,14]
        N_v_dash = mcmc[random_num,15]
        N_r_dash = mcmc[random_num,16]
        N_vvv_dash = mcmc[random_num,17]
        N_vvr_dash = mcmc[random_num,18]
        N_vrr_dash = mcmc[random_num,19]
        N_rrr_dash = mcmc[random_num,20]
        @NLconstraint(model, x[k+1,1] == x[k,1] + dt * ((X_H(x[k,1], x[k,2], x[k,3]) + X_R(x[k,1], x[k,2], x[k,3], δ_ref, n_p_ref) + X_P(x[k,1], x[k,2], x[k,3], δ_ref, n_p_ref) +  X_wind(x[k,1], x[k,2], x[k,6]) + u1[k,1] + (m + m_y) * x[k,2] * x[k,3]) / (m + m_x)))
        @NLconstraint(model, x[k+1,2] == x[k,2] + dt * ((Y_H(x[k,1], x[k,2], x[k,3]) + Y_R(x[k,1], x[k,2], x[k,3], δ_ref, n_p_ref) +  Y_wind(x[k,1], x[k,2], x[k,6]) + u2[k,1] - (m + m_x) * x[k,1] * x[k,3]) / (m + m_y)))
        @NLconstraint(model, x[k+1,3] == x[k,3] + dt * ((N_H(x[k,1], x[k,2], x[k,3]) + N_R(x[k,1], x[k,2], x[k,3], δ_ref, n_p_ref) +  N_wind(x[k,1], x[k,2], x[k,6]) + u3[k,1]) / (I_zG + J_z)))
        @NLconstraint(model, x[k+1,4] == x[k,4] + dt * (x[k,1] * cos(x[k,6]) - x[k,2] * sin(x[k,6])))
        @NLconstraint(model, x[k+1,5] == x[k,5] + dt * (x[k,1] * sin(x[k,6]) + x[k,2] * cos(x[k,6])))
        @NLconstraint(model, x[k+1,6] == x[k,6] + dt * (x[k,3]))
        @NLconstraint(model, x[k+1,7] == x[k+1,4] + L_f * cos(x[k+1,6]))
        @NLconstraint(model, x[k+1,8] == x[k+1,5] + L_f * sin(x[k+1,6]))
        @NLconstraint(model, x[k+1,9] == x[k+1,4] + L_t * cos(x[k+1,6] + pi))
        @NLconstraint(model, x[k+1,10] == x[k+1,5] + L_t * sin(x[k+1,6] + pi))
        # @NLconstraint(model, x[k+1,11] == x[k,11] + dt * (u1[k,1] - x[k,11]))
        # @NLconstraint(model, x[k+1,12] == x[k,12] + dt * (u2[k,1] - x[k,12]))
        # @NLconstraint(model, x[k+1,13] == x[k,13] + dt * (u3[k,1] - x[k,13]))
    end

    #評価関数の設定
    @NLobjective(model, Min, sum((Q*((x[k,7]-f_obs[k,10])^2 +(x[k,8]-f_obs[k,11])^2 +(x[k,9]-f_obs[k,12])^2 +(x[k,10]-f_obs[k,13])^2 + (x[t+N+1,7]-f_obs[t+N+1,10])^2 +(x[t+N+1,8]-f_obs[t+N+1,11])^2 +(x[t+N+1,9]-f_obs[t+N+1,12])^2 +(x[t+N+1,10]-f_obs[t+N+1,13])^2) + R1*((u1[k,1]-u1[k-1,1])^2) + R2*((u2[k,1]-u2[k-1,1])^2) + R3*((u3[k,1]-u3[k-1,1])^2)) for k in t+1:t+N))
    
    # 最適化の実行
    optimize!(model)
    
    # 次点の操作量x(t+1)とそれに用いた制御量u(t)の値を返す
    return value.(x[t+1,1:13]), value.(u1[t,1]), value.(u2[t,1]), value.(u3[t,1]) # 最適化後の入力x・制御量u
end

mpc (generic function with 1 method)

In [10]:
function run_mpc(duration::Int, dt::Float64, q::Int)
    global X_wind, Y_wind, N_wind
    global R_0_dash, X_vv_dash, X_vr_dash, X_rr_dash, X_vvvv_dash
    global Y_v_dash, Y_r_dash, Y_vvv_dash, Y_vvr_dash, Y_vrr_dash, Y_rrr_dash
    global N_v_dash, N_r_dash, N_vvv_dash, N_vvr_dash, N_vrr_dash, N_rrr_dash
    #観測データ
    obs = CSV.read("./observation/obs_EF.csv", DataFrame)
        
    # 初期値設定
    x_0 = Vector(Float64[0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    u1_0 = 0.0
    u2_0 = 0.0
    u3_0 = 0.0

    # データフレーム
    cols = Int(duration/dt) + 1
    df_x = DataFrame(
        time = collect(0:dt:duration),
        u = zeros(cols),
        v = zeros(cols),
        r = zeros(cols),
        x = zeros(cols),
        y = zeros(cols),
        ψ = zeros(cols),
        x1 = zeros(cols),
        y1 = zeros(cols),
        x2 = zeros(cols),
        y2 = zeros(cols),
        X_F_m = zeros(cols),
        Y_F_m = zeros(cols),
        N_F_m = zeros(cols),
        X_F = zeros(cols),
        Y_F = zeros(cols),
        N_F = zeros(cols),
        X_wind = zeros(cols),
        Y_wind = zeros(cols),
        N_wind = zeros(cols),
        U_W = zeros(cols),
        Ψ_W = zeros(cols)
    )

    #初期値をデータフレームへ
    df_x[1, 2:14] .= x_0[1:13]
    df_x[1, 15:17] .= [u1_0, u2_0, u3_0]
    df_x[1, 18:20] .=x_0[14:16]
    df_x[1, 21:22] .=[U_W, Ψ_W]
    # MPCの実行
    @showprogress for t in 1:Int(duration/dt)
        #MPCの結果
        x_val, u1_val, u2_val, u3_val = mpc(x_0, u1_0, u2_0, u3_0, obs, t, N)
        #結果をデータフレームへ
        df_x[t+1, 2:14] .= x_val
        df_x[t+1, 15:17] .= [u1_val, u2_val, u3_val]
        x_wind = X_wind(x_val[1], x_val[2], x_val[6])
        y_wind = Y_wind(x_val[1], x_val[2], x_val[6])
        n_wind = N_wind(x_val[1], x_val[2], x_val[6])
        df_x[t+1, 18:20] .= [x_wind, y_wind, n_wind]
        df_x[t+1, 21:22] .= [U_W, Ψ_W]
        
        #次のMPCに用いる初期値へ設定
        x_0 = Vector(x_val) 
        u1_0 = u1_val
        u2_0 = u2_val
        u3_0 = u3_val
    end

    # 結果書き出し
    CSV.write("./results/MPC_result$(q).csv", df_x)
end

run_mpc (generic function with 1 method)

In [ ]:
N = 30 # 予測ホライゾン
duration = 50 # 全体の計算時間
dt = 0.25 # 刻み幅
whole_step = Int(duration/dt) 

df = CSV.read("./observation/suzaku_windcoef.csv", DataFrame)
df = df[completecases(df), :]
Ψ_A_vec = deg2rad.(df[!,"psi_A[deg]"])
C_X_vec = df[!,"CX"]
C_Y_vec = df[!,"CY"]
C_N_vec = df[!,"CN"]
C_X_line = LinearInterpolation(Ψ_A_vec, C_X_vec)
C_Y_line = LinearInterpolation(Ψ_A_vec, C_Y_vec)
C_N_line = LinearInterpolation(Ψ_A_vec, C_N_vec)
function C_X(Ψ_A)
    return C_X_line(Ψ_A)
end

function C_Y(Ψ_A)
    return C_Y_line(Ψ_A)
end

function C_N(Ψ_A)
    return C_N_line(Ψ_A)
end

A_F = 0.183313 * L_pp * B 
A_L = 3.143061 * L_pp * d

U_W = 2.0
Ψ_W = 0.5 * pi

# U_W_list = U_W .* ones(Float64, 1:whole_step)
# Ψ_W_list = Ψ_W .* ones(Float64, 1:whole_step)

for q in 1:100
    run_mpc(duration, dt, q)
end

Progress:   1%|▍                                        |  ETA: 0:19:16


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



Progress:   5%|██                                       |  ETA: 0:09:05